<a href="https://colab.research.google.com/github/shkimusw/bigdata/blob/main/%5B12%EC%A3%BC%EC%B0%A8_%EA%B0%95%EC%9D%98%5D_%EB%8C%80%EC%A4%91%EA%B5%90%ED%86%B5_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#대중교통 데이터로 상관분석과 회귀분석하기

구글에 '티머니 교통통계자료' 라고 검색해서 나오는 첫 링크에 들어가서 가장 최신 파일을 다운받아보자.

In [ ]:
#글자깨짐 문제 수정을 위해 다음의 코드 실행 
#단계 1: 폰트 설치
import matplotlib.font_manager as fm
!apt-get -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()


# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)


In [1]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False
	
# 한글 폰트 설정
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_name = fm.FontProperties(fname=path, size=18).get_name()
plt.rc('font', family=font_name)
fm._rebuild()

In [4]:
from google.colab import files
uploaded=files.upload()

Saving subwayfee.csv to subwayfee.csv


In [5]:
import pandas as pd

In [ ]:
subway_data = pd.read_csv('subwayfee.csv', encoding = 'cp949', thousands=',')
subway_data

데이터 훑어보기

In [ ]:
# 데이터프레임의 기본정보
subway_data.info()

In [ ]:
# 통계정보 요약
subway_data.describe()

원하는 통계치 구하기

In [ ]:
print(subway_data['유임승차'] / subway_data['무임승차'])
print('')
print(subway_data['유임승차'] / (subway_data['유임승차'] + subway_data['무임승차']))

In [ ]:
#최대값 구하기
(subway_data['유임승차'] / subway_data['무임승차']).max()

In [ ]:
(subway_data['유임승차'] / (subway_data['유임승차'] + subway_data['무임승차'])).max()

In [ ]:
#유임승차 비율을 rate이라는 변수명으로 생성해보자
rate_table = pd.DataFrame(subway_data['유임승차'] / (subway_data['유임승차'] + subway_data['무임승차']), columns = ['rate'])
rate_table

위에서 만든 데이터프레임을 subway_data 데이터프레임에 새로운 열로 추가한 뒤 출력해 보자.
concat 함수를 사용한다. axis=1 옵션은 옆으로 붙여주는 옵션.

In [ ]:
added_subway = pd.concat([subway_data, rate_table], axis = 1)
added_subway

In [ ]:
#승하차 총 인원 열 추가
added_subway['총이용객']=added_subway.loc[:, ['유임승차', '유임하차', '무임승차', '무임하차']].sum(axis = 1)
added_subway

사용자가 가장 많은 역 찾기

In [ ]:
added_subway[added_subway['총이용객'] == added_subway['총이용객'].max()]

#상관관계 히트맵

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(data=added_subway.corr(), square=True, annot=True)
plt.show()

#회귀분석
가장 기본적인 OLS 모델을 돌려보자.

종속변수를 유임승차율, 독립변수를 총이용객으로 놓는다.

In [17]:
from statsmodels.formula.api import ols

In [19]:
res = ols('rate ~ 총이용객', data=added_subway).fit()

In [ ]:
#결과 확인
res.summary()

계수값이 작으므로 총이용객을 로그로 변환한다.

In [21]:
import numpy as np

In [22]:
added_subway['loguser']=np.log(added_subway['총이용객'])

In [ ]:
added_subway

다시 OLS를 돌려본다.

In [ ]:
res = ols('rate ~ loguser', data=added_subway).fit()
res.summary()